In [1]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [2]:
from git import Repo

In [3]:
def getCommitsFrom(project):
    projectPath = "repositories\\"+project
    repository = Repo(projectPath)
    repository.git.checkout("master", "-f")
    iterCommits = repository.iter_commits()
    commits = []
    for c in iterCommits:
        commits.append(c)
    return commits

In [4]:
def checkoutTo(project, sha):
    projectPath = "repositories\\" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [5]:
import os

In [6]:
def extractMetricsWithUnderstand(owner, project):
    understandPath = "understand\\SciTools\\bin\\pc-win64\\und"
    os.system('cmd /c "{0} create -languages java {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} add {1} {2}"'.format(understandPath, "repositories\\"+owner+"\\"+project, project))
    os.system('cmd /c "{0} settings -metrics all {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} settings -metricsOutputFile {1}.csv {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} -quiet analyze {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} metrics {1}"'.format(understandPath, project))

In [7]:
import pandas as pd

In [8]:
def sumMetricsPerSample(owner, project):
    print(project)
    dataframe = pd.read_csv(project+".csv")
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Public Class"]
    dataframe["sample"] = owner+"\\"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["numberJavaFiles"] = numberJavaFiles
    return dataframe

In [9]:
import fnmatch

In [10]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    metrics = sumMetricsPerSample(owner, project)
    return metrics

In [11]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [12]:
pip install JPype1

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Boiler plate stuff to start the module
import jpype
from jpype import *
from statistics import mean

class Readability(object):
    def __init__(self, project):
        super(Readability, self).__init__()
        self.project = project
        self.startJVM()
        self.readabilityPackage = JPackage("raykernel").apps.readability.eval.Main
        self.repositoryPath = "repositories\\"+project

    def startJVM(self):
        if not jpype.isJVMStarted():
            jpype.startJVM(jpype.getDefaultJVMPath(), '-Djava.class.path=readability.jar',  '-ea', '-Xmx4096M', convertStrings=True)
#             jpype.startJVM(classpath="readability.jar", convertStrings=True)


    def shutdownJVM(self):
        jpype.shutdownJVM()

    def getReadability(self):
        javaFiles = findPaths("*.java", self.repositoryPath)
        if len(javaFiles) == 0: return 0
        readability = 0
        array = []
        for javaFile in javaFiles:
            try:
                file = open(javaFile).read()
            except:
                file = ""
            array.append(float(self.readabilityPackage.getReadability(file)))
        return mean(array)

In [14]:
def getReadability(owner, project):
    r = Readability(owner+"\\"+project)
    readability = r.getReadability()
    del r
    return readability

In [15]:
def getMetrics(commit, owner, project):
    metrics = getUnderstandMetrics(owner, project)
    metrics["commitSha"] = commit.hexsha
    metrics["commitDate"] = commit.authored_datetime
    metrics["readability"] = getReadability(owner, project)
    return metrics

In [16]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".csv")
    os.remove(sample + ".udb")

In [17]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [18]:
def replaceSamplePathForWindowsLike(sample):
    return sample.replace("/", "\\")

In [19]:
def extractMetricsByCommit(sample, commit):
    owner, project = sample.split("\\")
    checkoutTo(sample, commit.hexsha)
    print("commit ======= " + commit.hexsha)
    metrics = getMetrics(commit, owner, project)
    return metrics

In [20]:
def extractMetricsForAllCommits(commits, sample):
    allCommits = pd.DataFrame()
    for index, commit in enumerate(commits):
        metrics = extractMetricsByCommit(sample, commit)
        allCommits = allCommits.append(metrics, ignore_index=True)
        print("{0}% of commits completed from sample {1}".format((index/len(commits) * 100), sample))
    return allCommits

In [21]:
def metricsByCommit(framework, samples):
    
    for index, sample in enumerate(samples):
        sample = replaceSamplePathForWindowsLike(sample)
        
        printStatus(index+1, len(samples))
        
        createDirectoryIfNotExists("metrics")
        
        commits = getCommitsFrom(sample)
        commits.reverse()
        
        owner, project = sample.split("\\")
        
        createDirectoryIfNotExists("metrics\\"+owner)

        ########## é so rodar, esta com a hash certa para o proximo
        # for index, commit in enumerate(commits):
        #     if commit.hexsha == "dfe62cb3e72c7a9cfd759dc7411197d9a629f813":
        #         position = index
        # commits = commits[position+1:]
        
        allCommits = extractMetricsForAllCommits(commits, sample)
        
        deleteUnusedFiles(project)
        
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)

In [22]:
androidSamples = pd.read_csv("..\\1-GettingQuestions\\samplesWithQuestions\\androidSamples.csv")
awsSamples = pd.read_csv("..\\1-GettingQuestions\\samplesWithQuestions\\awsSamples.csv")
azureSamples = pd.read_csv("..\\1-GettingQuestions\\samplesWithQuestions\\azureSamples.csv")
springSamples = pd.read_csv("..\\1-GettingQuestions\\samplesWithQuestions\\springSamples.csv")

In [23]:
len(awsSamples["path"])

8

In [24]:
len(azureSamples["path"])

31

In [25]:
len(springSamples["path"])

3

In [26]:
len(androidSamples["path"])

80

In [27]:
metricsByCommit("Spring", springSamples["path"])

33.33333333333333% Completed samples
commit ======= 1dccd558464d83d6aed95d7eed28b13b1730dada
gs-accessing-data-neo4j
0.0% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 5e9e3c66d1bdbd99d836a8e2299f51593ef45af0
gs-accessing-data-neo4j
0.45045045045045046% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 56a2c69e08309db765bb7cd0b671b85db2403f4b
gs-accessing-data-neo4j
0.9009009009009009% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 08d8b4ac921c5199adb7fca93e000c4e85b37601
gs-accessing-data-neo4j
1.3513513513513513% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= de2438314a55fd33495535acf8902a717d175fa6
gs-accessing-data-neo4j
1.8018018018018018% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= e73972f6c6ec6f11d32a3e08af4a74241602f663
gs-accessing-data-neo4j
2.2522522522522523% of commits completed f

gs-accessing-data-neo4j
21.62162162162162% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 0c4517e8f985ec17574e01ec884352e4dc38d61f
gs-accessing-data-neo4j
22.07207207207207% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 9f450fa55a48a010bcdb61ce842e93103da097ac
gs-accessing-data-neo4j
22.52252252252252% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 4897500b08ce77aec52c8d423442075438c8a75c
gs-accessing-data-neo4j
22.972972972972975% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= db45b08cd6b7e19c7106ed9df389695466f9d0e7
gs-accessing-data-neo4j
23.423423423423422% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 3e7c1db518fec03ad4ab0bd650093b85b72664f1
gs-accessing-data-neo4j
23.873873873873876% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= f9dd5cde54ed3882a6

gs-accessing-data-neo4j
43.69369369369369% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= ce20d87d3c5079c095335127205c8fb3bb013727
gs-accessing-data-neo4j
44.14414414414414% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 08f5caf7f27d57019476409a6e24328bb4be0486
gs-accessing-data-neo4j
44.5945945945946% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 9498b13c66b2f3db93965990874aa7b9c87082ed
gs-accessing-data-neo4j
45.04504504504504% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 3475b9e7f16b97677b0d23c3ef42c72f7ee27b61
gs-accessing-data-neo4j
45.4954954954955% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= fa114719fba1828f7241ca02d4f56ab7c4eec0b0
gs-accessing-data-neo4j
45.94594594594595% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 96ea25076e949a0d906fef2

gs-accessing-data-neo4j
65.76576576576578% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 31947ad4d3760209f2ef6d77568738664568808e
gs-accessing-data-neo4j
66.21621621621621% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= abd968085962427511d269f736f4a71a813a80a5
gs-accessing-data-neo4j
66.66666666666666% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= d8c85bdd775f708d7945f825d8488ea3ca86811c
gs-accessing-data-neo4j
67.11711711711712% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 7f1d462a53854fa2ca7c32b5fa5427f302b50c62
gs-accessing-data-neo4j
67.56756756756756% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 435c5b1b1a7a49415398ee4e0eefd13593cebe60
gs-accessing-data-neo4j
68.01801801801803% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 785f452bc09c50bb8ef8a

gs-accessing-data-neo4j
87.83783783783784% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 8ed42573ddb1c71e7027189f4dd5da8c2ff55171
gs-accessing-data-neo4j
88.28828828828829% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 0f00ff1fe828017be6172b03ba3cbe8066b7f3a1
gs-accessing-data-neo4j
88.73873873873875% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= e0b2e91ae325e1d7aa0c2b896079e47481857d4d
gs-accessing-data-neo4j
89.1891891891892% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 327896279c09d0eee46c410f597dade8028f3c36
gs-accessing-data-neo4j
89.63963963963964% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= fc6319d8ad923cd6f5856cea757de2eb224fa0ec
gs-accessing-data-neo4j
90.09009009009009% of commits completed from sample spring-guides\gs-accessing-data-neo4j
commit ======= 639ae111cbba4111e5a3cd

...

gs-accessing-data-mongodb
33.33333333333333% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= b0fd61530d90fb54764593bfeb5a56aa4f9e1c4f
gs-accessing-data-mongodb
33.82352941176471% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 81425abe1f51f2c4fde99d97ae4952109d37b558
gs-accessing-data-mongodb
34.31372549019608% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 8e78f29d6b2efd3271f46f987cc200205e36a82e
gs-accessing-data-mongodb
34.80392156862745% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 73a17b80392999322a44d1d5e5fef58e2ec5a66d
gs-accessing-data-mongodb
35.294117647058826% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= be5d9d862fa9b4126df995554ee03e5234e00e8f
gs-accessing-data-mongodb
35.78431372549019% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ====

gs-accessing-data-mongodb
56.86274509803921% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 891f71212932ae11dc949288c1a09b1f40b03e9f
gs-accessing-data-mongodb
57.35294117647059% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= b82264fc992d5d1507e565f9dd7ac557a68efcb4
gs-accessing-data-mongodb
57.84313725490197% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= e89163e40c380990acc5a51db1f528c9deca41ac
gs-accessing-data-mongodb
58.333333333333336% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 46dab029cd85fd0f3eccf2c40f7eff07550e8802
gs-accessing-data-mongodb
58.82352941176471% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 68523fdc44c38525e5b17514d21c8e8fdf133ccf
gs-accessing-data-mongodb
59.31372549019608% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ====

gs-accessing-data-mongodb
80.3921568627451% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 021d02e519d170336ab15c2f57f670c47be3fc0c
gs-accessing-data-mongodb
80.88235294117648% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= a7d3c3158fdd3242e8809d3062a8aca54246123a
gs-accessing-data-mongodb
81.37254901960785% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= ec19657ca78740fbe18d6a287396aa4663f8c62e
gs-accessing-data-mongodb
81.86274509803921% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 0ccc1db5bf114d33441d7aa42df6f0c2a28b70b9
gs-accessing-data-mongodb
82.35294117647058% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======= 4c73713058ef95aa60e46f1c8001e7085e502e7d
gs-accessing-data-mongodb
82.84313725490196% of commits completed from sample spring-guides\gs-accessing-data-mongodb
commit ======

commit ======= 77884690cdc711e111c3fba023ec37c341ca37fc
tut-spring-boot-oauth2
7.8431372549019605% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= baf5c545e5f8dbfdfb40b53c1d27e57b816df63c
tut-spring-boot-oauth2
8.823529411764707% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 15fb4c06177b475b7f1f990b9ca955f762494659
tut-spring-boot-oauth2
9.803921568627452% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 3d379ec8139fcf24e79bf48666c19aa70d61b16c
tut-spring-boot-oauth2
10.784313725490197% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 8bed06e0144266ae8de2ca242e0293d93cc7dd84
tut-spring-boot-oauth2
11.76470588235294% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 54593b598c78604d3a537de05df362e411c8177e
tut-spring-boot-oauth2
12.745098039215685% of commits completed from sample spring-guides\tut-spring-b

tut-spring-boot-oauth2
55.88235294117647% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 7a3e0aa98d35ec021b807644b0b901efb10bac89
tut-spring-boot-oauth2
56.86274509803921% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 930f9b8e7f8d7c58e57477090dcee5deb3a1075a
tut-spring-boot-oauth2
57.84313725490197% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 8bf0e320ca63c6f9f3833ae9bc28d18d44699905
tut-spring-boot-oauth2
58.82352941176471% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= 4730692f3dd47906a29e47cb7bb5b34585b01f2d
tut-spring-boot-oauth2
59.80392156862745% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= e09007d24c7e829d51e203d22640da77aa02e3f2
tut-spring-boot-oauth2
60.78431372549019% of commits completed from sample spring-guides\tut-spring-boot-oauth2
commit ======= a277f299ac044b40286a52a434a898a7c

In [27]:
metricsByCommit("Android", androidSamples["path"])
metricsByCommit("AWS", awsSamples["path"])

12.5% Completed samples
commit ======= 542bcd83a059c4e4901a8566095bab8a5bc076af
amazon-kinesis-learning
0.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= d4dd87fb23d4912ca7bb0abfc89367c582a35efc
amazon-kinesis-learning
5.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= 181ee5830b4401b54e64c9a784ea2973d96e381a
amazon-kinesis-learning
10.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= c9ec5c6f3e85738210fd7d05e903b9fcab27b702
amazon-kinesis-learning
15.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= 16d2207f75d48ce938543385cc61b3b658f2d4a7
amazon-kinesis-learning
20.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= 30d69726d34a4afe850c77e845e29401c2d74732
amazon-kinesis-learning
25.0% of commits completed from sample aws-samples\amazon-kinesis-learning
commit ======= 94c22d04f0645c2ec1d3b857d04877e31

aws-cognito-java-desktop-app
6.0606060606060606% of commits completed from sample aws-samples\aws-cognito-java-desktop-app
commit ======= 6034e6012ba3892e53473e0cd73e9182c9169119
aws-cognito-java-desktop-app
9.090909090909092% of commits completed from sample aws-samples\aws-cognito-java-desktop-app
commit ======= 5d3cb2ad1f7959a0a4335067c8c9eb4335d28199
aws-cognito-java-desktop-app
12.121212121212121% of commits completed from sample aws-samples\aws-cognito-java-desktop-app
commit ======= d975c3345c8b577c25653f7836191d66db45b440
aws-cognito-java-desktop-app
15.151515151515152% of commits completed from sample aws-samples\aws-cognito-java-desktop-app
commit ======= 3f88cf61f2ddb1f0fb93c84af3183cfbf5744b29
aws-cognito-java-desktop-app
18.181818181818183% of commits completed from sample aws-samples\aws-cognito-java-desktop-app
commit ======= 3fd46f2202e36c1028fb1251f936bba259242ba4
aws-cognito-java-desktop-app
21.21212121212121% of commits completed from sample aws-samples\aws-cognito-j

aws-cloudhsm-jce-examples
23.52941176470588% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= b8b6398e5a7f98d3431553ca7e119f20011a9ef9
aws-cloudhsm-jce-examples
25.0% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 486f951616508f3d7fd341c1dd8d4e3d552e03ce
aws-cloudhsm-jce-examples
26.47058823529412% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 332b35165e9d5b395976838fede937ff758620fb
aws-cloudhsm-jce-examples
27.941176470588236% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 407a435315f963f34262ad8ffbcd452601ed2664
aws-cloudhsm-jce-examples
29.411764705882355% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= b14c49fe7c7c887d4e589e1c340026b3ed6e5ff7
aws-cloudhsm-jce-examples
30.88235294117647% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 4942bb67f84feb775d09

aws-cloudhsm-jce-examples
94.11764705882352% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 6bdc3399f9be56815a3ccd5073c510b60cbfcabf
aws-cloudhsm-jce-examples
95.58823529411765% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= ba0e485a5c6573bc19c81bb27ce92fbbeaef9d96
aws-cloudhsm-jce-examples
97.05882352941177% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
commit ======= 62a90892459a0f6a9e231e7b790e459691167acd
aws-cloudhsm-jce-examples
98.52941176470588% of commits completed from sample aws-samples\aws-cloudhsm-jce-examples
62.5% Completed samples
commit ======= 92a33634ad41aec230e5fa2ea9f92490b3283889
amazon-ecs-java-microservices
0.0% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= 15dfaeace77af3fcaaaef065d70c999a9620aaf4
amazon-ecs-java-microservices
1.7241379310344827% of commits completed from sample aws-samples\amazon-ecs-java-microservic

amazon-ecs-java-microservices
72.41379310344827% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= d5803457d3276e4c5ae174fcb65aad625cd16228
amazon-ecs-java-microservices
74.13793103448276% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= 99dfe3f6e67e319243ed6dcc8f37c91ba10ed575
amazon-ecs-java-microservices
75.86206896551724% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= 4d0db1d228dd994cf834b3439d318acd9fdeacff
amazon-ecs-java-microservices
77.58620689655173% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= b0fb4729d36c35c70f42d9f4e1e8bb2a8be43f16
amazon-ecs-java-microservices
79.3103448275862% of commits completed from sample aws-samples\amazon-ecs-java-microservices
commit ======= c9f71ead0a80dbe28dbc2ba671c1a93ee2be8ee5
amazon-ecs-java-microservices
81.03448275862068% of commits completed from sample aws-samples\amazon-

aws-transcribe-streaming-example-java
28.57142857142857% of commits completed from sample aws-samples\aws-transcribe-streaming-example-java
commit ======= 94dd7748658eb11506f5db03e3ee4688e0890e16
aws-transcribe-streaming-example-java
32.142857142857146% of commits completed from sample aws-samples\aws-transcribe-streaming-example-java
commit ======= 2e6e1c07c2a0818f0ad46788c8f341d701135f55
aws-transcribe-streaming-example-java
35.714285714285715% of commits completed from sample aws-samples\aws-transcribe-streaming-example-java
commit ======= f8f1aeb2bd479cef6da96d9676d5b4c47ad1e978
aws-transcribe-streaming-example-java
39.285714285714285% of commits completed from sample aws-samples\aws-transcribe-streaming-example-java
commit ======= 11829f8eb9bd5b9ea9397d48182193018c028e3a
aws-transcribe-streaming-example-java
42.857142857142854% of commits completed from sample aws-samples\aws-transcribe-streaming-example-java
commit ======= 3a0b71d0e379cae2d4208f34893fe5f16ebd996a
aws-transcribe-s

In [27]:
metricsByCommit("Azure", azureSamples["path"])

9.090909090909092% Completed samples
commit ======= bcfd2fa0cfa5aa64851df5952c3a61a8b4d80a18
cognitive-services-qnamaker-java
0.0% of commits completed from sample Azure-Samples\cognitive-services-qnamaker-java
commit ======= 26a8ec7d6dd7f14895a0cc1032155eb0683a0e45
cognitive-services-qnamaker-java
2.631578947368421% of commits completed from sample Azure-Samples\cognitive-services-qnamaker-java
commit ======= d0c11d8052a42f6e3e1e30b0a324fb60e14b8383
cognitive-services-qnamaker-java
5.263157894736842% of commits completed from sample Azure-Samples\cognitive-services-qnamaker-java
commit ======= b4f8c8d4de4c0965c49c9e75875f6f1934610653
cognitive-services-qnamaker-java
7.894736842105263% of commits completed from sample Azure-Samples\cognitive-services-qnamaker-java
commit ======= a8cb82bfaa13e2f2f9d73e4a65c100c2f94871d7
cognitive-services-qnamaker-java
10.526315789473683% of commits completed from sample Azure-Samples\cognitive-services-qnamaker-java
commit ======= 0dd8c52c3477f1c548547

active-directory-b2c-android-native-appauth
18.51851851851852% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-appauth
commit ======= 189adfb79caaa6f81ab3035cf77e39c224e40446
active-directory-b2c-android-native-appauth
22.22222222222222% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-appauth
commit ======= 21f90094d40300b3b3bcc614602adf190f08e1cc
active-directory-b2c-android-native-appauth
25.925925925925924% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-appauth
commit ======= a3b69ce21d99d024ac675965d2d73e8bbff91305
active-directory-b2c-android-native-appauth
29.629629629629626% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-appauth
commit ======= 3178095328ce5f9a9a50b6dc689c347c39a65372
active-directory-b2c-android-native-appauth
33.33333333333333% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-appauth
comm

azure-cosmos-db-graph-java-getting-started
51.42857142857142% of commits completed from sample Azure-Samples\azure-cosmos-db-graph-java-getting-started
commit ======= b744c4756d7191ec7bda3d4abc00fb617918b931
azure-cosmos-db-graph-java-getting-started
54.285714285714285% of commits completed from sample Azure-Samples\azure-cosmos-db-graph-java-getting-started
commit ======= 47ea3464f5b84cf7e5e86a30d32765a1081435b5
azure-cosmos-db-graph-java-getting-started
57.14285714285714% of commits completed from sample Azure-Samples\azure-cosmos-db-graph-java-getting-started
commit ======= f48f5ec1859cc311a0359f87328fd3b53d371b59
azure-cosmos-db-graph-java-getting-started
60.0% of commits completed from sample Azure-Samples\azure-cosmos-db-graph-java-getting-started
commit ======= c16f0a2f3dbd3ccad7120b6015e8d583d7de94a5
azure-cosmos-db-graph-java-getting-started
62.857142857142854% of commits completed from sample Azure-Samples\azure-cosmos-db-graph-java-getting-started
commit ======= ec56db2f99d3

0.0% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 743f40fbcddffb66cef058b082f569c8555376fb
active-directory-b2c-android-native-msal
1.9230769230769231% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 829d97e5f9e5717a8a6634de0cc1ea1f2f950f3d
active-directory-b2c-android-native-msal
3.8461538461538463% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 9aca62c1f70ceee3259ae5a792c0393e22cdb756
active-directory-b2c-android-native-msal
5.769230769230769% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= cae43e9f0f7ba6e547fb7e382185d2523f51491a
active-directory-b2c-android-native-msal
7.6923076923076925% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 25081e29235cf95b3f8134b6b9fbe061a4150f55
active-directory-b2c-android-nat

active-directory-b2c-android-native-msal
78.84615384615384% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 8ad772041212477967db621cf1fcf6f346f0489b
active-directory-b2c-android-native-msal
80.76923076923077% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= a5ddb409a14b4e4968ea6793f24e7d292fc0bbab
active-directory-b2c-android-native-msal
82.6923076923077% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= fe73fffa4d4440676c9ca9d31647042a617ea488
active-directory-b2c-android-native-msal
84.61538461538461% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= 87a684cee598527ac939619dfc9587f3fd41f310
active-directory-b2c-android-native-msal
86.53846153846155% of commits completed from sample Azure-Samples\active-directory-b2c-android-native-msal
commit ======= dcca7b0523bcf8472442fe

storage-blob-java-getting-started
69.56521739130434% of commits completed from sample Azure-Samples\storage-blob-java-getting-started
commit ======= 072d737ec87cff53892d3e13be158a4bb5af775a
storage-blob-java-getting-started
71.73913043478261% of commits completed from sample Azure-Samples\storage-blob-java-getting-started
commit ======= 20d8fc3dfd33ce700c86c15668b73d1868ecabfc
storage-blob-java-getting-started
73.91304347826086% of commits completed from sample Azure-Samples\storage-blob-java-getting-started
commit ======= 80d390baed2084153e3218296b86775ef0f06677
storage-blob-java-getting-started
76.08695652173914% of commits completed from sample Azure-Samples\storage-blob-java-getting-started
commit ======= 683b8225a32c71b0b0d30f3a3cf3aabf34bde1bb
storage-blob-java-getting-started
78.26086956521739% of commits completed from sample Azure-Samples\storage-blob-java-getting-started
commit ======= 19d1b99c302cd1f4fbcf7f4d1b1d1b0351528503
storage-blob-java-getting-started
80.4347826086956

storage-table-java-getting-started
78.37837837837837% of commits completed from sample Azure-Samples\storage-table-java-getting-started
commit ======= 6d12e15a031ea14b59286eb854fb486826f053c1
storage-table-java-getting-started
81.08108108108108% of commits completed from sample Azure-Samples\storage-table-java-getting-started
commit ======= 27565ff200488b188f959f3c71de04db0342b9c5
storage-table-java-getting-started
83.78378378378379% of commits completed from sample Azure-Samples\storage-table-java-getting-started
commit ======= 5a6922515b303d3fc44d5b1a4e08a1c713fb0be7
storage-table-java-getting-started
86.48648648648648% of commits completed from sample Azure-Samples\storage-table-java-getting-started
commit ======= eb442c58d0fe7fbd657b53830fbfba4df1c59ff7
storage-table-java-getting-started
89.1891891891892% of commits completed from sample Azure-Samples\storage-table-java-getting-started
commit ======= 2da6e9124f82f4cf87ad860521f67eb9621fab29
storage-table-java-getting-started
91.891